In [ ]:
#### Вариант 2:
**"Анализ продаж"**

- Создать DataFrame из sales.csv- Найти топ-5 продуктов по выручке- 
Рассчитать среднюю цену по регонам
- 
Сохранить результаты в Parquet

In [19]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, LongType, StringType, IntegerType, DecimalType, TimestampType
from pyspark.sql.functions import col, sum, round, avg

spark = SparkSession.builder \
        .appName("Lab1_v2") \
        .master("local[*]") \
        .config("spark.sql.shuffle.partitions", 4) \
        .getOrCreate()

file_path = "../../data/sales.csv"

data_schema = StructType([
    StructField("order_id", LongType(), False),
    StructField("product", StringType(), False),
    StructField("quantity", IntegerType(), True),
    StructField("price", DecimalType(19, 14), True),
    StructField("customer_id", LongType(), False),
    StructField("order_date", TimestampType(), True),
    StructField("region", StringType(), True)
])

df_sales = spark.read.csv(
    file_path,
    header=True,
    schema=data_schema
)

In [20]:
df_sales.show(5)

+--------+-------+--------+-------------------+-----------+-------------------+------+
|order_id|product|quantity|              price|customer_id|         order_date|region|
+--------+-------+--------+-------------------+-----------+-------------------+------+
|       1|Speaker|       4|1719.23452186000850|        170|2025-08-16 00:00:00|  West|
|       2|  Phone|       4| 148.36607033775795|        202|2025-06-04 00:00:00|  West|
|       3| Tablet|       6| 897.50964873086780|        490|2025-02-18 00:00:00| North|
|       4| Laptop|       3|  34.37911222099545|        398|2025-03-23 00:00:00| South|
|       5|Monitor|       9|1161.59350706994770|        418|2024-11-15 00:00:00|  West|
+--------+-------+--------+-------------------+-----------+-------------------+------+
only showing top 5 rows



In [21]:
# Топ-5 продуктов по выручке
# df_top5 = df_sales.orderBy(col("price"), ascending=False).limit(5)
df_top5 = df_sales.groupBy(col("product")).agg(
    round(sum(col("price")), 2).alias("Total sum")
).limit(5)
df_top5.show()

+--------+----------+
| product| Total sum|
+--------+----------+
|Keyboard|1864773.72|
| Speaker|1820395.89|
|   Mouse|1883975.16|
|   Phone|1931162.05|
|  Tablet|1889764.36|
+--------+----------+



In [25]:
# Рассчитать среднюю цену по регионам

df_avg = df_sales.groupBy(col("region")).agg(
    round(avg(col("price")), 2).alias("Avg price")
).orderBy(col("Avg price").desc())
df_avg.show()


+------+---------+
|region|Avg price|
+------+---------+
| North|  1012.87|
| South|  1006.00|
|  West|   999.39|
|  East|   998.72|
+------+---------+



In [26]:
# Сохранить результаты в Parquet

output_base_path = "../../data/lab01/v1"

df_top5.write \
    .mode("overwrite") \
    .format("parquet") \
    .save(f"{output_base_path}/df_top5")

df_avg.write \
    .mode("overwrite") \
    .format("parquet") \
    .save(f"{output_base_path}/df_avg")

print(f"Датафреймы сохранены в {output_base_path}")

Датафреймы сохранены в ../../data/lab01/v1
